<a href="https://colab.research.google.com/github/ajaynain-eng/2D-to-3D-CNN-Pivotal-Teleradiology/blob/main/Data_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
 import os
 os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [13]:
!kaggle datasets download -d ymirsky/medical-deepfakes-lung-cancer 

100% 5.98G/5.99G [01:27<00:00, 74.9MB/s]
100% 5.99G/5.99G [01:27<00:00, 73.5MB/s]


In [ ]:
!unzip \*.zip && rm *.zip

In [16]:
!pip install pydicom

In [17]:
# importing supporting libraries
import pandas as pd                    # to create and read data frame and series
import numpy as np                     # to work with array
from matplotlib import pyplot as plt   # to plot figures
import seaborn as sns                  # to plot interactive metric plot and heatmaps
import cv2                             # to read and show image
from skimage.transform import resize   # as we need to resize the image to reduce model complexity and get optimum accuracy

from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score

import os
import warnings
import pydicom as dicom
import glob
import scipy.ndimage
from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection


# importing required CNN algorithms, optimizers and activation function
import tensorflow as tf
from tensorflow import keras
#from keras.applications import VGG16
from keras import Sequential
from keras.layers import Conv1D, Conv2D, Conv3D, MaxPool3D, ZeroPadding3D, Dropout, BatchNormalization
from keras.layers import Flatten, Dense, Input
from keras.layers import LeakyReLU
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, SGD
from keras.models import Model
from keras import callbacks
# importing pretrained model VGG16 and VGG19
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19


from skimage import measure
from skimage.transform import resize
from matplotlib.projections import Axes3D
from plotly.figure_factory import create_trisurf
from mpl_toolkits import mplot3d

In [18]:
def transform_to_hu(medical_image, image):
    intercept = medical_image.RescaleIntercept
    slope = medical_image.RescaleSlope
    hu_image = image * slope + intercept
    return hu_image

def window_image(image, window_center, window_width):
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    window_image = image.copy()
    window_image[window_image < img_min] = img_min
    window_image[window_image > img_max] = img_max
    return window_image

def crop_image(image, display=False):
# Create a mask with the background pixels
    mask = image == 0
# Find the lung area
    coords = np.array(np.nonzero(~mask))
    top_left = np.min(coords, axis=1)
    bottom_right = np.max(coords, axis=1)
# Remove the background
    croped_image = image[top_left[0]:bottom_right[0],
                         top_left[1]:bottom_right[1]]
    return croped_image

def no_crop(medical_img):         # changes the medical image in pixel array and returns the original image
    return medical_img.pixel_array

def crop_the_image_1(medical_img):     # crop the image corresponding to HU range for lungs
    temp_medical_img = medical_img
    temp_img = temp_medical_img.pixel_array
    hu_image = transform_to_hu(temp_medical_img,temp_img)
    lung_image = window_image(hu_image, -500, 1500)
    croped_img = crop_image(lung_image, display=False)
    return croped_img

In [19]:
def get_data(pid, crop_function):         #  Selects and crops 110 slices and give 3D and 2D image data
    files = glob.glob("/content/CT_Scans/EXP1_blind/"+ pid +"/*.dcm")
    slices = [dicom.read_file(i) for i in files]
    slices = sorted(slices, key=lambda s: s.SliceLocation)
    s = 0
    views = np.empty([110,110,110])
    x = (len(slices)-4)/109      # Select 110 slices
    for i in range(110):
        t = int(4+(x*i))
        try:
            t_img = crop_function(slices[t])
            t_img = resize(t_img, (110,110))
        except:
            t_img = crop_function(slices[t-1])
            t_img = resize(t_img, (110,110))
            s = s + 1
        views[:,:,i] = t_img          # 3D image array
        i += 1
    img_2d = views[55,:,:].T
    img_2d = img_2d[::-1]           # 2D image array
    print(s)     
    return views, img_2d

In [20]:
def get_correlation(data, threshold):
    corr_col = set()
    corr = data.corr()
    for i in range(len(corr.columns)):
        for j in range(i):
            if abs(corr.iloc[i,j]) > threshold:  #if abs(corr.iloc[i,j]) > threshold:
                corr_col.add(corr.columns[i])
    return corr_col

def get_uncorr_data(train_data, test_data, threshold):

    corr_columns = get_correlation(pd.DataFrame(train_data), threshold)


    corrmat = pd.DataFrame(train_data).corr()

    corrdata = corrmat.stack().sort_values(ascending = False)  #corrdata = corrmat.abs().stack().sort_values(ascending = False)

    corrdata = corrdata[corrdata > threshold]
    corrdata = corrdata[corrdata < 1]
    corrdata = pd.DataFrame(corrdata).reset_index()
    corrdata.columns = ['feature1','feature2', 'value']

    grouped_column_list = []
    correlated_group_list = []
    for column in corrdata['feature1']:
        if column not in grouped_column_list:
            correlated_block = corrdata[corrdata.feature1 == column]
            grouped_column_list = grouped_column_list + list(correlated_block.feature2.unique()) + [column]
            correlated_group_list.append(correlated_block)

    important_columns = []
    for data in correlated_group_list:
        columns = list(data.feature1.unique()) + list(data.feature2.unique())
        important_columns.append(columns[0])

    new_to_drop = list(corr_columns - set(important_columns))

    l = new_to_drop

    columns_to_drop = list()
    for i in l:
        if i not in [0,1,108,109]:
            if (i-1 not in columns_to_drop and i+1 not in columns_to_drop):     ####
                if (i-2 not in columns_to_drop and i+2 not in columns_to_drop):
                    columns_to_drop.append(int(i))

    col_to_drop_df = pd.concat([corrdata, pd.Series(columns_to_drop, name = 'column_to_drop')], axis = 1)
    
    return col_to_drop_df

In [21]:
def get_cross_val_score(col_to_drop, imp_col, y_0):    
    rf_score_mean = list()      # diff_1 = droped_col[i] - droped_col[i-1]                             
    knn_score_mean = list()     # diff_2 = droped_col[i+1] - droped_col[i] - 1
    knn_neighbors = list()      # training_col = imp_col[diff_1:-diff_2]
    neighbors = 2
    estimators = 60
    for i, j in enumerate(col_to_drop):
        rf = RandomForestRegressor(n_estimators=40, random_state=0)
        rf_score_mean.append(np.array(cross_val_score(rf, pd.DataFrame(y_0[0])[imp_col[i]] + y_0[0][:,j:j+1], pd.DataFrame(y_0[0])[j])).mean()*100)
        knn_val_score = cross_val_score(KNeighborsRegressor(n_neighbors=neighbors), pd.DataFrame(y_0[0])[imp_col[i]] + y_0[0][:,j:j+1], pd.DataFrame(y_0[0])[j])
        mean = knn_val_score.mean()*100
        m = list()
        n = list()
        p = list()
        if mean < 0.95:
            for k, o in enumerate([4, 6, 8, 10,12, 24]):
                scr = cross_val_score(KNeighborsRegressor(n_neighbors=o), pd.DataFrame(y_0[0])[imp_col[i]] + y_0[0][:, j:j+1], pd.DataFrame(y_0[0])[j])
                m.append(o)
                n.append(scr.mean())
                p.append(scr)
            max_arg = np.array(n).argmax()
            knn_neighbors.append(m[max_arg])
            mean = n[max_arg]*100
        else:
            knn_neighbors.append(2)
        knn_score_mean.append(mean)
    return  rf_score_mean, knn_score_mean, knn_neighbors

In [23]:
def get_model_to_predict_col(y_train_0, y_test_0, y0):      # here y0 = y_train_0 + y_test_0   (i.e complete data)
    all_slices_knn_score = list()
    all_slices_rf_score = list()
    len_sel_col = list()           
    len_droped_col = list()
    y_train = np.empty((len(y_train_0),110,88,110))
    y_test = np.empty((len(y_test_0),110,88,110))
    for s in range(0,110):
        y_0 = y_train_0[:,:,:,s]
        y_test0 = y_test_0[:,:,:,s]
        col_to_drop_df = get_uncorr_data(y_0[5], y_test0[5], 0.96)
        col_to_drop = list(col_to_drop_df.dropna().sort_values(by = 'value', ascending=False)['column_to_drop'])
        col_to_drop = [int(i) for i in col_to_drop]
        col_to_drop.sort()
        len_sel_col.append(len(col_to_drop))

        imp_col = list()
        for i in np.arange(2,108):
            if i in col_to_drop:
                imp_col.append([i-2,i-1,i+1,i+2])
        rf_score_mean, knn_score_mean, knn_neighbors = get_cross_val_score(col_to_drop, imp_col, y0[:,:,:,s])
        score_df = pd.DataFrame({'col_to_drop' : col_to_drop, 'rf_score_mean' : rf_score_mean, 'knn_score_mean' : knn_score_mean})
        droped_col = list(score_df.sort_values(by='knn_score_mean', ascending=False)['col_to_drop'])[:22]
        droped_col.sort()
        print(len(droped_col))
        all_slices_knn_score.append(np.mean(knn_score_mean))
        all_slices_rf_score.append(np.mean(rf_score_mean))

        #y_train[:,:,:s] = pd.DataFrame(y_0).drop(droped_col, axis = 1)
        #y_test[:,:,:s] = pd.DataFrame(y_test0).drop(droped_col, axis = 1)
        for z in range(len(y_train_0)):
            y_train[z,:,:,s] = np.array(pd.DataFrame(y_0[z]).drop(droped_col, axis = 1)).reshape(110,88)
        for z in range(len(y_test0)):
            y_test[z,:,:,s] = np.array(pd.DataFrame(y_test0[z]).drop(droped_col, axis = 1)).reshape(110,88)

    return y_train, y_test, all_slices_knn_score, all_slices_rf_score

In [24]:
def plot_learning_curve(model, epochs):   
    plt.figure(figsize=(15,6))
    plt.plot(np.arange(0,epochs), model.history['accuracy'], color = 'green', label = 'train_accuracy')
    plt.plot(np.arange(0,epochs), model.history['loss'], color = 'red', label = 'train_loss')
    plt.plot(np.arange(0,epochs), model.history['val_accuracy'], color = 'blue', label = 'test_accuracy')
    plt.plot(np.arange(0,epochs), model.history['val_loss'], color = 'black', label = 'test_loss')
    plt.grid()
    plt.xlabel('no. of epochs')
    plt.ylabel('accuracy and loss')
    plt.legend(loc = 'lower right')

def plot_accuracy_curve(model, epochs):    
    plt.figure(figsize=(15,6))
    plt.plot(np.arange(0,epochs), model.history['accuracy'], color = 'green', label='accuracy')
    plt.plot(np.arange(0,epochs), model.history['val_accuracy'], color = 'red', label = 'test accuracy')
    plt.grid()
    plt.legend(loc = 'lower right')

In [25]:
patients_id = os.listdir('/content/CT_Scans/EXP1_blind')

In [28]:
f = patients_id.copy()
g2d_0 = np.zeros((80,110,110))
views_0 = np.zeros((80,110,110,110))
fcount_0 = []
for i,j in enumerate(f):
    print(i)
    img_3d, img_2d = get_data(j, no_crop)
    g2d_0[i] = img_2d
    views_0[i] = img_3d
    #views_0[i] = selected_slices(j, no_crop)
x_0 = g2d_0
y_0 = views_0
np.save('Data_out_0.npy', views_0)              # Temprory storing the data in drive
np.save('Data_in_0.npy', g2d_0)

0
1
1
1
2
1
3
1
4
0
5
1
6
1
7
1
8
1
9
1
10
1
11
1
12
1
13
1
14
1
15
1
16
1
17
1
18
1
19
1
20
1
21
1
22
1
23
1
24
1
25
1
26
1
27
1
28
1
29
1
30
1
31
1
32
1
33
1
34
1
35
1
36
1
37
1
38
1
39
1
40
1
41
1
42
1
43
1
44
0
45
1
46
0
47
1
48
1
49
1
50
1
51
1
52
0
53
1
54
1
55
1
56
1
57
1
58
1
59
0
60
1
61
1
62
1
63
1
64
1
65
1
66
1
67
1
68
1
69
1
70
1
71
1
72
1
73
1
74
1
75
1
76
0
77
1
78
1
79
1


In [29]:
f = patients_id.copy()
g2d_1 = np.zeros((80,110,110))
views_1 = np.zeros((80,110,110,110))
for i,j in enumerate(f):
    print(i)
    img_3d, img_2d = get_data(j, crop_the_image_1)
    g2d_1[i] = img_2d
    views_1[i] = img_3d
    #views_0[i] = selected_slices(j, no_crop)
x_1 = g2d_1
y_1 = views_1
np.save('Data_out_1.npy', views_1)
np.save('Data_in_1.npy', g2d_1)

0
1
1
1
2
1
3
1
4
0
5
1
6
1
7
1
8
1
9
1
10
1
11
1
12
1
13
1
14
1
15
1
16
1
17
1
18
1
19
1
20
1
21
1
22
1
23
1
24
1
25
1
26
1
27
1
28
1
29
1
30
1
31
1
32
1
33
1
34
1
35
1
36
1
37
1
38
1
39
1
40
1
41
1
42
1
43
1
44
0
45
1
46
0
47
1
48
1
49
1
50
1
51
1
52
0
53
1
54
1
55
1
56
1
57
1
58
1
59
0
60
1
61
1
62
1
63
1
64
1
65
1
66
1
67
1
68
1
69
1
70
1
71
1
72
1
73
1
74
1
75
1
76
0
77
1
78
1
79
1


In [30]:
x_0 = np.load('/content/Data_in_0.npy')
y_0 = np.load('/content/Data_out_0.npy')
x_1 = np.load('/content/Data_in_1.npy')
y_1 = np.load('/content/Data_out_1.npy')

In [31]:
x_train_0 = x_0[:66]
x_test_0 = x_0[65:]
y_train_0 = y_0[:66]
y_test_0 = y_0[65:]

x0 = x_train_0[:,1,:]
y0 = y_train_0[:,:,:,1]          # temp_x = x_0[:][:,1,:], temp_y = y_0[:][:,:,:,1]
x_test0 = x_test_0[:,1,:]
y_test0 = y_test_0[:,:,:,1]
x_train_1 = x_1[:66]
x_test_1 = x_1[65:]
y_train_1 = y_1[:66]
y_test_1 = y_1[65:]

x1 = x_train_1[:,1,:]
y1 = y_train_1[:,:,:,1]
x_test1 = x_test_1[:,1,:]
y_test1 = y_test_1[:,:,:,1]

In [32]:
x_test0 = x_test0.reshape(15,110,1)
x0 = x0.reshape(66,110,1)

x_test1 = x_test1.reshape(15,110,1)
x1 = x1.reshape(66,110,1)

x0.shape, y0.shape

((66, 110, 1), (66, 110, 110))

In [33]:
x_train_0.shape

(66, 110, 110)

In [35]:
y_train.shape, y_test.shape

((66, 110, 88, 110), (15, 110, 88, 110))

In [36]:
x = np.empty((66,110,110,3))
for i in range(3):
    x[:,:,:,i] = x_train_0


y = y_train
x.shape, y.shape

((66, 110, 110, 3), (66, 110, 88, 110))

In [38]:
vgg_16 = VGG16(input_shape=(110,110,3), weights='imagenet', include_top=False)
vgg_16.summary()

58900480/58889256 [==============================] - 1s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 110, 110, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 110, 110, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 110, 110, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 55, 55, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 55, 55, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 55, 55, 128)       147584    
______________________________________________________________

In [39]:
vgg_16 = VGG16(input_shape=(110,110,3), weights='imagenet', include_top=False)
for layer in vgg_16.layers[:10]:
    layer.trainable = False
input = vgg_16.layers[:5]
model = Sequential()
model.add(Conv2D(filters=3, kernel_size=(3,3), padding='same', activation='relu', input_shape = (110,110,3)))    #LeakyReLU(alpha=1)
model.add(input[0])
model.add(input[1])
model.add(input[2])


model.add(Conv2D(filters=110, kernel_size=(1,23), padding="valid", activation='relu'))
model.add(Conv2D(filters=110, kernel_size=(3,3), padding="same", activation='relu'))
#model.add(x[3])
#model.add(x[4])
#model = Model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 110, 110, 3)       84        
_________________________________________________________________
input_2 (InputLayer)         multiple                  0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 110, 110, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 110, 110, 64)      36928     
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 110, 88, 110)      162030    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 110, 88, 110)      109010    
Total params: 309,844
Trainable params: 271,124
Non-trainable params: 38,720
_____________________________________________

In [40]:
model.compile(loss = 'mean_squared_error', optimizer= 'adam', metrics = ['accuracy']) 
history = model.fit(x, y, batch_size=32, epochs = 20)

Epoch 1/20
3/3 [==============================] - 42s 7s/step - loss: 0.8144 - accuracy: 0.0057
Epoch 2/20
3/3 [==============================] - 26s 7s/step - loss: 0.0413 - accuracy: 0.0057
Epoch 3/20
3/3 [==============================] - 26s 7s/step - loss: 0.0047 - accuracy: 0.0062
Epoch 4/20
3/3 [==============================] - 30s 8s/step - loss: 0.0021 - accuracy: 0.1401
Epoch 5/20
3/3 [==============================] - 26s 7s/step - loss: 0.0019 - accuracy: 0.1497
Epoch 6/20
3/3 [==============================] - 26s 7s/step - loss: 0.0019 - accuracy: 0.1549
Epoch 7/20
3/3 [==============================] - 26s 7s/step - loss: 0.0018 - accuracy: 0.1586
Epoch 8/20
3/3 [==============================] - 26s 7s/step - loss: 0.0018 - accuracy: 0.1602
Epoch 9/20
3/3 [==============================] - 26s 7s/step - loss: 0.0017 - accuracy: 0.1649
Epoch 10/20
3/3 [==============================] - 26s 7s/step - loss: 0.0017 - accuracy: 0.1618
Epoch 11/20
3/3 [======================